In [ ]:
import numpy as np
import sys

In [ ]:
# Link Parameters:    [a, alpha, d, theta]
# For Multiple Links: [[a, alpha, d, theta]
#                      [a, alpha, d, theta]
#                      [a, alpha, d, theta]]

def atransformation(LinkParameters):
    a = LinkParameters[0]
    d = LinkParameters[2]
    alpha = LinkParameters[1]
    theta = LinkParameters[3]
    A = np.array([[[np.cos(theta), -np.sin(theta) * np.cos(alpha), np.sin(theta) * np.sin(alpha), a * np.cos(theta)],
                   [np.sin(theta), np.cos(theta) * np.cos(alpha), -np.cos(theta) * np.sin(alpha), a * np.sin(theta)],
                   [0, np.sin(alpha), np.cos(alpha), d],
                   [0, 0, 0, 1]]])
    return (A)


# Homogenous Matrices Forward Kinematics from DH
def forward_kinematics_homogenous_matrix(DH):
    A = np.identity(4)
    n = np.shape(DH)[0]

    for i in range(n):
        Anext = atransformation(DH[i])
        A = np.matmul(A,Anext)

    return A


In [ ]:
# Importing Libraries
import numpy as np
import matplotlib.pyplot as plt

# Creating Class Robot and defining functions
class Robot():
  def __init__(self, numberoflinks, DH_matrix,joint_types=-1):
    self.numberoflinks = numberoflinks
    self.DH_matrix = DH_matrix
    if(joint_types == -1):
      self.joint_types = [1]*numberoflinks
    else:
      self.joint_types = joint_types

  def transform_matrix(self):
    self.A = []
    self.R = []
    for i in range(self.numberoflinks):
      self.A.append(np.array([[np.cos(self.DH_matrix[i][0]), -np.sin(self.DH_matrix[i][0]) * np.cos(self.DH_matrix[i][3]), np.sin(self.DH_matrix[i][0]) * np.sin(self.DH_matrix[i][3]), self.DH_matrix[i][2] * np.cos(self.DH_matrix[i][0])],[np.sin(self.DH_matrix[i][0]), np.cos(self.DH_matrix[i][0]) * np.cos(self.DH_matrix[i][3]), -np.cos(self.DH_matrix[i][0]) * np.sin(self.DH_matrix[i][3]), self.DH_matrix[i][2] * np.sin(self.DH_matrix[i][0])],[0, np.sin(self.DH_matrix[i][3]), np.cos(self.DH_matrix[i][3]), self.DH_matrix[i][1]],[0, 0, 0, 1]])) 
      self.R.append(np.array([[np.cos(self.DH_matrix[i][0]), -np.sin(self.DH_matrix[i][0]) * np.cos(self.DH_matrix[i][3]), np.sin(self.DH_matrix[i][0]) * np.sin(self.DH_matrix[i][3])],[np.sin(self.DH_matrix[i][0]), np.cos(self.DH_matrix[i][0]) * np.cos(self.DH_matrix[i][3]), -np.cos(self.DH_matrix[i][0]) * np.sin(self.DH_matrix[i][3])],[0, np.sin(self.DH_matrix[i][3]), np.cos(self.DH_matrix[i][3])]]))
    self.A = np.array(self.A) # Array of Homogenous Transformation Matrices
    self.R = np.array(self.R) # Array of Rotation matrices
    
    # Creating array of R_0_n matrices (R01, R02, R03, R04 ...)
    self.R_0_n = []
    A = np.identity(3)
    for i in range(self.numberoflinks):
      self.R_0_n.append(A)
      A = np.matmul(A, self.R[i])
    self.R_0_n.append(A)
    self.R_0_n = np.array(self.R_0_n)

    # Creating array of H_0_n matrices (H01, H02, H03, H04 ...)
    self.H_0_n = []
    B = np.identity(4)
    for i in range(self.numberoflinks):
      self.H_0_n.append(B)
      B = np.matmul(B, self.A[i])
    self.H_0_n.append(B)
    self.H_0_n = np.array(self.H_0_n)
    
    # Calculating Final Transfornation Matrix
    T = np.asmatrix(np.identity(4))
    for j in range(self.numberoflinks):
        T = T * np.asmatrix(self.A[j]) #,dtype = 'int' for integer output
    self.T = T
    # print ("A Matrices are: \n", self.A)
    # print ("Final Transformation Matrix: \n",T)
    # print ("Rotation Matrices are: \n", self.R)
    # print ("Rotation Matrices w.r.t base frame are: \n", self.R_0_n)
  
  def manipulator_jacobian(self):
    # Creating z terms
    self.z = []
    k = np.array([[0],[0],[1]])
    for i in range(self.numberoflinks+1):
      self.z.append(np.matmul(self.R_0_n[i],k))
    self.z = np.array(self.z)

    # Creating O terms
    d = np.array([[0],[0],[0],[1]])
    self.O = np.matmul(self.H_0_n,d)[:self.numberoflinks+1]
    self.On = np.delete(self.O,(3),axis=1)

    # Creating Jacobian Manipulator (Here J1 will be 0th element of array J and so on)
    self.J = []
    for i in range(self.numberoflinks):
      if self.joint_types[i] == 1:
        X = self.On[-1] - self.On[i]
        J_v = np.cross(self.z[i], X,axis = 0)
        J_v = np.vstack((J_v, self.z[i]))
        self.J.append(J_v)
      else:
        J_v = np.vstack((self.z[i], np.array([[0],[0],[0]])))
        self.J.append(J_v)
    self.J = np.array(self.J)
    self.J_v = np.delete(self.J, (3,4,5), axis = 1)
    # print(self.J.shape)
    print ("Manipulator Jacobian: \n", self.J)
    # print ("End effector position: \n", self.On[-1])
    # print ("End effector velocity: \n", "v = ", self.J_v, " x q_dot")

  def joint_velocities(self, xdot, ydot, zdot):
    X_dot  =np.array([[xdot],[ydot],[zdot]])
    # print(self.J_v.shape)
    J_v = self.J_v.reshape((3,3))
    Jacobian_Rank = np.linalg.matrix_rank(J_v)
    Augmentedmatrix_Rank = np.linalg.matrix_rank(np.hstack((J_v, X_dot)))
    if Jacobian_Rank != Augmentedmatrix_Rank:
      print("ERROR: Velocities provided are not possible for the given configuration")
      # sys.exit()
    else:
      q_dot = np.linalg.pinv(J_v) @ X_dot
      print("Joint Velocities are given by: \n", q_dot)

# Initializing object "robot" and calculating result
# Mention the number of links in the first input
# Enter the DH table row wise in the order: [THETA, D, A, ALPHA]
# Enter Rho values for each link: 1 for Revolute and 0 for Prismatic

# robot = Robot(2,[[0, 0, 1, 0],[0, 0, 1, 0]],joint_types = [0,0])
# robot.transform_matrix()
# robot.manipulator_jacobian()

In [ ]:
# scara
def scara_invkin(x,y,z,d1,d2):
    # using formulae from the textbook
    r = abs((x**2+y**2-d1**2-d2**2)/(2*d1*d2))
    theta2 = np.arctan(np.sqrt(abs(1-r**2))/r)
    theta1 = np.arctan(y/x) - np.arctan((d2*np.sin(theta2))/(d1+d2*np.cos(theta2)))
    d3 = -z
    # converting from radians to degrees
    theta1_deg = theta1*180/np.pi
    theta2_deg = theta2*180/np.pi
    print(f"theta1 = {theta1_deg} deg")
    print(f"theta2 = {theta2_deg} deg")
    return theta1, theta2

In [ ]:
# coordinates of the block

a = (0.40, 0.05, 0.1)
b = (0.40, 0.04, 0.1)
c = (0.40, 0.03,0.1)
d = (0.40, 0.02, 0.1)

# length of the links 
d1 = 0.25
d2 = 0.25

# joint angle for point a
print("joint angles for point A:")
theta1, theta2 = scara_invkin(a[0],a[1],a[2],d1, d2)

# DH parameters for joint angles at A
# a = 0
# d = 0
# alpha = 0
# theta = np.pi/4
# LinkParameters = [a, alpha, d, theta]
LinkParameters = [[0.25, np.pi/2, 0, theta1],[0.25, -np.pi/2, 0, theta2], [-a[2], 0, 0, 0]]
scara_robot = Robot(3,LinkParameters,[1, 1, 0])
scara_robot.transform_matrix()
scara_robot.manipulator_jacobian()
scara_robot.joint_velocities(a[0],a[1],a[2])



# joint angle for point b
print("\njoint angles for point B:")
theta1, theta2 = scara_invkin(b[0],b[1],b[2],d1, d2)
LinkParameters = [[0.25, np.pi/2, 0, theta1],[0.25, -np.pi/2, 0, theta2], [-b[2], 0, 0, 0]]
scara_robot = Robot(3,LinkParameters,[1, 1, 0])
scara_robot.transform_matrix()
scara_robot.manipulator_jacobian()
scara_robot.joint_velocities(b[0],b[1],b[2])

# joint angle for point c
print("\njoint angles for point C:")
theta1, theta2 = scara_invkin(c[0],c[1],c[2],d1, d2)
LinkParameters = [[0.25, np.pi/2, 0, theta1],[0.25, -np.pi/2, 0, theta2], [-c[2], 0, 0, 0]]
scara_robot = Robot(3,LinkParameters,[1, 1, 0])
scara_robot.transform_matrix()
scara_robot.manipulator_jacobian()
scara_robot.joint_velocities(c[0],c[1],c[2])


# joint angle for point d
print("\njoint angles for point D:")
theta1, theta2 = scara_invkin(d[0],d[1],d[2],d1, d2)
LinkParameters = [[0.25, np.pi/2, 0, theta1],[0.25, -np.pi/2, 0, theta2], [-d[2], 0, 0, 0]]
scara_robot = Robot(3,LinkParameters,[1, 1, 0])
scara_robot.transform_matrix()
scara_robot.manipulator_jacobian()
scara_robot.joint_velocities(d[0],d[1],d[2])



joint angles for point A:
theta1 = -29.146182089237147 deg
theta2 = 72.54239687627789 deg
Manipulator Jacobian: 
 [[[ 7.41256634e-01]
  [ 1.89273893e-01]
  [-0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 1.00000000e+00]]

 [[ 1.11022302e-16]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [-1.20495503e-01]
  [ 4.71898630e-01]
  [ 8.73379939e-01]]

 [[ 3.92239777e-01]
  [-5.82196248e-01]
  [ 7.12176584e-01]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]]]
ERROR: Velocities provided are not possible for the given configuration

joint angles for point B:
theta1 = -30.77657560095997 deg
theta2 = 72.97433747691923 deg
Manipulator Jacobian: 
 [[[ 7.78776309e-01]
  [ 1.98854239e-01]
  [-0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 1.00000000e+00]]

 [[ 1.11022302e-16]
  [ 0.00000000e+00]
  [ 1.38777878e-17]
  [-1.26594540e-01]
  [ 4.95784397e-01]
  [ 8.59169165e-01]]

 [[ 3.89067794e-01]
  [-5.67539248e-01]
  [ 7.25620737e-01]
  [ 0.00000000e+00]
  [ 0.0000